In [1]:
"""Generate data batch."""
import os

import pandas as pd
import numpy as np
import tables
from sklearn.utils import shuffle

In [2]:
def get_train_batch(self):
    """Get [train_batch_size] examples as one train batch. Both question and answer
    are converted to int. The data batch is selected from short buckets to long buckets."""
    vgg_batch = []
    c3d_batch = []
    question_batch = []
    answer_batch = []

    bucket = self.train_buckets[self.current_bucket]
    start = self.train_batch_idx[
        self.current_bucket] * self.train_batch_size
    end = start + self.train_batch_size

    question_encode = bucket.iloc[start:end]['question_encode'].values
    answer_batch = bucket.iloc[start:end]['answer_encode'].values
    video_ids = bucket.iloc[start:end]['video_id'].values
    batch_length = self.train_batch_length[self.current_bucket]

    #print(self.video_feature.root.vgg.shape, self.train_batch_size)
    for i in range(len(question_encode)):
        qid = [int(x) for x in question_encode[i].split(',')]
        qid = np.pad(qid, (0, batch_length - len(qid)), 'constant')
        question_batch.append(qid)
        #print(max(video_ids))
        #print(type(video_ids[i]), video_ids[i])
        vgg_batch.append(self.video_feature.root.vgg[video_ids[i] - 1])
        c3d_batch.append(self.video_feature.root.c3d[video_ids[i] - 1])

    self.train_batch_idx[self.current_bucket] += 1
    # if current bucket is ran out, use next bucket.
    # print('batch_idx', self.train_batch_idx[self.current_bucket])
    # print('current_bucket', self.current_bucket)
    # print(self.train_batch_total)
    if self.train_batch_idx[self.current_bucket] == self.train_batch_total[self.current_bucket]:
        self.current_bucket += 1
    while self.current_bucket != len(self.train_batch_total) \
          and self.train_batch_total[self.current_bucket] == 0:
        # if current bucket is empty --> next bucket
        self.current_bucket += 1
    if self.current_bucket == len(self.train_batch_total):
        self.has_train_batch = False
    #print(len(question_batch))
    # answer to one-hot like array
    answer_onehot = np.zeros((len(answer_batch), self.answer_num), dtype=np.int64)  
    for i, row in enumerate(answer_batch):
        for a in row:
            answer_onehot[i][a] = 1    

    return vgg_batch, c3d_batch, question_batch, answer_onehot



In [3]:
x=[1.5,2.2]
x=[int(x) for x in x]

In [4]:
x

[1, 2]

In [5]:
question_encode=["1.5,2.2","6.5,2.2"]
q=[x.split(',') for x in question_encode]

In [6]:
q

[['1.5', '2.2'], ['6.5', '2.2']]

In [7]:
[[int(float(item)) for item in x] for x in q]

[[1, 2], [6, 2]]

In [8]:
int(str('1.5'))

ValueError: invalid literal for int() with base 10: '1.5'

In [ ]:
int('1.5')

In [9]:
def get_val_example(self):
    """Get one val example. Only question is converted to int."""
    start = self.val_example_idx * self.train_batch_size
    end = start + self.train_batch_size

    question_encode = self.val_qa.iloc[start:end]['question_encode']
    video_id = self.val_qa.iloc[start:end]['video_id']
    answer = self.val_qa.iloc[start:end]['answer']

    #question = [int(x) for x in question_encode.split(',')]
    question = [x.split(',') for x in question_encode]
    question = [[int(item) for item in x] for x in question]
    vgg = self.video_feature.root.vgg[video_id - 1]
    c3d = self.video_feature.root.c3d[video_id - 1]

    self.val_example_idx += 1
    if self.val_example_idx == self.val_example_total:
        self.has_val_example = False

    answer_onehot = np.zeros((self.train_batch_size, self.answer_num), dtype=np.int64)
    for i, ans_ in enumerate(answer):
        for ans in ans_:
            answer_onehot[i, self.answer_dict[ans]] = 1

    return vgg, c3d, question, answer_onehot

def get_test_example(self):
    """Get one test example. Only question is converted to int."""
    start=self.test_example_idx*self.train_batch_size
    end=start+self.train_batch_size
    example_id = self.test_qa.iloc[start:end]['id']
    question_encode = self.test_qa.iloc[
        start:end]['question_encode']
    video_id = self.test_qa.iloc[start:end]['video_id']
    answer = self.test_qa.iloc[start:end]['answer']

    #question = [int(x) for x in question_encode.split(',')]
    question = [x.split(',') for x in question_encode]
    question=[[int(float(item)) for item in x] for x in question]
    vgg = self.video_feature.root.vgg[video_id]
    c3d = self.video_feature.root.c3d[video_id]

    self.test_example_idx += self.train_batch_size
    if self.test_example_idx == self.test_example_total:
        self.has_test_example = False

    return vgg, c3d, question, answer, example_id


In [10]:
import time
start = time.time()
# np.sum(np.eye(4000)[[1,2]], axis=0)
# a = np.zeros([1,4000])
# b = [1, 2000,3999]
# for line in b:
#     for i in range(3999):
#         a[0][i] = 1
print(time.time() - start)

2.0265579223632812e-05


In [11]:
from sklearn.preprocessing import OneHotEncoder

In [12]:
enc = OneHotEncoder()
enc.fit([[0, 0, 3], [1, 1, 0], [0, 2, 1], [1, 0, 2]]) 
print ("enc.n_values_ is:",enc.n_values_)
print ("enc.feature_indices_ is:",enc.feature_indices_)
print (enc.transform([[0, 1, 1]]).toarray())

enc.n_values_ is: [2 3 4]
enc.feature_indices_ is: [0 2 5 9]
[[1. 0. 0. 1. 0. 0. 1. 0. 0.]]


In [13]:
enc = OneHotEncoder()
enc.fit([[0, 0, 3], [0, 1, 0], [0, 2, 1], [0, 0, 2]]) 
print ("enc.n_values_ is:",enc.n_values_)
print ("enc.feature_indices_ is:",enc.feature_indices_)
print (enc.transform([[0, 1, 1]]).toarray())

enc.n_values_ is: [1 3 4]
enc.feature_indices_ is: [0 1 4 8]
[[1. 0. 1. 0. 0. 1. 0. 0.]]


In [14]:
enc = OneHotEncoder()
enc.fit([[0, 0, 3], [1, 0, 3], [0, 0, 4], [2, 0, 3]]) 
print ("enc.n_values_ is:",enc.n_values_)
print ("enc.feature_indices_ is:",enc.feature_indices_)
print (enc.transform([[0, 1, 1]]).toarray())

enc.n_values_ is: [3 1 5]
enc.feature_indices_ is: [0 3 4 9]


ValueError: unknown categorical feature present [1] during transform.